In [ ]:
# SISO 5G gNB-UE Simulation using AIRSTRAN D 2200

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

# Other imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", category=UserWarning, module="jupyter_client")

# Import relevant components from Sionna RT
from sionna.rt import load_scene, Transmitter, Receiver, Camera, PathSolver
from sionna.rt import AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Load empty scene
scene = load_scene("/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml")

In [ ]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

In [ ]:
# Install plyfile if not available
try:
    from plyfile import PlyData
except ImportError:
    import os
    os.system("pip install plyfile")
    from plyfile import PlyData

In [ ]:
# ============================================
# SISO Configuration: gNB to UE
# ============================================

# Set the operating frequency (n48 band for 5G)
scene.frequency = 3.7e9  # 3.7 GHz

# Define UE position (fixed to start)
ue_position = [10.0, 0.0, 0.0]   # UE position (x, y, z in meters)

# ============================================
# Antenna Configuration
# ============================================

# gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
gnb_pattern = gnb_pattern_factory(polarization="V")

# UE antenna: Isotropic pattern (typical for mobile devices)
ue_pattern_factory = antenna_pattern_registry.get("iso")
ue_pattern = ue_pattern_factory(polarization="V")

# SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

# Configure antenna arrays
scene.tx_array = AntennaArray(
    antenna_pattern=gnb_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

scene.rx_array = AntennaArray(
    antenna_pattern=ue_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

# ============================================
# Add Receiver to Scene
# ============================================

# Create UE receiver
rx = Receiver(name="ue", position=ue_position, display_radius=0.03)
scene.add(rx)

# ============================================
# Configure Propagation Environment
# ============================================

# Disable scattering for basic simulation
for radio_material in scene.radio_materials.values():
    radio_material.scattering_coefficient = 0.0

In [ ]:
from scene_parser import extract_building_info
from tx_placement import TxPlacement

In [ ]:
# ============================================
# Place gNB on a Specific Building
# ============================================

scene_xml_path = "/home/tingjunlab/Development/geo2sigmap/scenes/Duke_Perkins/scene.xml"
building_info = extract_building_info(scene_xml_path, verbose=False)

selected_building_id = 5  # Change this to your desired building number

# TxPlacement will create the transmitter if it doesn't exist and place it on the building
# Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=3.5)
tx_placer.set_rooftop_center()

# Get reference to the transmitter (already added to scene by TxPlacement)
tx = tx_placer.tx
# Convert to flat numpy array instead of nested list
gnb_position = tx.position.numpy().flatten().tolist()

# Point antenna toward UE
tx.look_at(ue_position)

print(f"\nSuccess! gNB placed on building {selected_building_id}")
print(f"Position: {gnb_position}")

## NLOS Box Area Test Case

In [ ]:
from boresight_pathsolver import create_zone_mask

map_config_nlos = {
    'center': [gnb_position[0], gnb_position[1], 0],
    'size': [500, 500],
    'cell_size': (1, 1),
    'ground_height': 0.0,
}

zone_params_nlos = {
    'center': (-115, 115),
    'width': 50,
    'height': 50
}

nlos_zone_mask, nlos_naive_look_at, nlos_zone_stats = create_zone_mask(
    map_config=map_config_nlos,
    zone_type='box',
    zone_params=zone_params_nlos,
    target_height=1.5,
    scene_xml_path=scene_xml_path,
    exclude_buildings=True
)

print("NLOS Zone Mask Created:")
print(f"  Naive Look-at: {nlos_naive_look_at}")
print(f"  Zone Centroid: {nlos_zone_stats['centroid_xy']}")

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(
    nlos_zone_mask,
    origin='lower',
    cmap='RdYlGn',
    extent=[
        map_config_nlos['center'][0] - map_config_nlos['size'][0]/2,
        map_config_nlos['center'][0] + map_config_nlos['size'][0]/2,
        map_config_nlos['center'][1] - map_config_nlos['size'][1]/2,
        map_config_nlos['center'][1] + map_config_nlos['size'][1]/2,
    ],
    vmin=0,
    vmax=1
)
plt.colorbar(im, ax=ax, label='Zone Mask (1.0 = in zone, 0.0 = out)')
ax.plot(gnb_position[0], gnb_position[1], 'r*', markersize=20, label='TX')
ax.set_title('NLOS Box Coverage Zone')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
from boresight_pathsolver import optimize_boresight_pathsolver

nlos_best_boresight, nlos_loss_hist, nlos_bore_hist, nlos_grad_hist, nlos_cov_stats = optimize_boresight_pathsolver(
    scene=scene,
    tx_name="gnb",
    map_config=map_config_nlos,
    scene_xml_path=scene_xml_path,
    zone_mask=nlos_zone_mask,
    initial_boresight=nlos_naive_look_at.tolist(),
    num_sample_points=100,
    learning_rate=2.0,
    num_iterations=100,
    loss_type='coverage_maximize',
    power_threshold_dbm=-75,
    verbose=True,
    seed=33
)

In [ ]:
from boresight_pathsolver import compare_boresight_performance

fig_nlos, nlos_comparison_stats = compare_boresight_performance(
    scene=scene,
    tx_name="gnb",
    map_config=map_config_nlos,
    zone_mask=nlos_zone_mask,
    naive_boresight=nlos_naive_look_at,
    optimized_boresight=nlos_best_boresight,
    title="NLOS Boresight Optimization Results"
)

plt.show()

print(f"NLOS Mean improvement: {nlos_comparison_stats['improvement_mean_dB']:.2f} dB")
print(f"NLOS Median improvement: {nlos_comparison_stats['improvement_median_dB']:.2f} dB")